# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules

In [2]:
data = pd.read_csv("lastfm.csv")
data

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


In [3]:
data = data[data.country.isin(['Germany', 'Netherlands'])]
data

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289611,19695,feist,f,Germany
289612,19695,eels,f,Germany
289613,19695,scissor sisters,f,Germany
289614,19695,editors,f,Germany


## Предобработка данных
### Поиск пропусков

In [4]:
data.isnull().sum()

user       0
artist     0
sex        0
country    0
dtype: int64

In [5]:
for col in data.columns:
    print(col, data[col].unique())

user [    1    31    33 ... 19681 19685 19695]
artist ['red hot chili peppers' 'the black dahlia murder' 'goldfrapp' ...
 'built to spill' 'caetano veloso' 'o.s.t.r.']
sex ['f' 'm']
country ['Germany' 'Netherlands']


Похоже, с колонками user, sex и country все в порядке. Изучим отдельно колонку artist

In [6]:
data.artist.value_counts()

coldplay                 296
red hot chili peppers    267
die Ärzte                247
linkin park              245
rammstein                228
                        ... 
o.s.t.r.                   1
coma                       1
happysad                   1
los hermanos               1
guided by voices           1
Name: artist, Length: 1002, dtype: int64

In [7]:
(data.artist.str.lower() == '0').sum()

0

In [8]:
(data.artist.str.lower() == '-1').sum()

0

In [9]:
(data.artist.str.lower() == 'unknown').sum()

0

In [10]:
(data.artist.str.lower() == '[unknown]').sum() #похоже, пропущенное значение

95

In [11]:
data.loc[:,'artist'] = data['artist'].replace('[unknown]', np.NaN)

c:\users\danil\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [12]:
data.isna().mean()

user       0.0000
artist     0.0028
sex        0.0000
country    0.0000
dtype: float64

Поскольку пропущенных значений меньше процента, мы можем позволить себе их удалить

In [13]:
data = data.dropna() 

In [14]:
data.isnull().sum()

user       0
artist     0
sex        0
country    0
dtype: int64

### Удаление дубликатов

In [15]:
data.drop_duplicates()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289611,19695,feist,f,Germany
289612,19695,eels,f,Germany
289613,19695,scissor sisters,f,Germany
289614,19695,editors,f,Germany


## Поиск правил

In [16]:
print(data['country'].value_counts());

Germany        24169
Netherlands     9660
Name: country, dtype: int64


In [17]:
data_by_artists = data.groupby(['user','sex', 'country'])['artist'].apply(lambda x: ','.join(x)).reset_index()
data_by_artists

,user,sex,country,artist
0,1,f,Germany,"red hot chili peppers,the black dahlia murder,..."
1,31,f,Netherlands,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
2,33,f,Germany,"death cab for cutie,tegan and sara,kimya dawso..."
3,42,f,Germany,"soundtrack,groove coverage,avril lavigne,the r..."
4,51,f,Germany,"kate nash,arctic monkeys,lykke li,adele,tegan ..."
...,...,...,...,...
1718,19662,m,Germany,"blind guardian,in extremo,subway to sally,finn..."
1719,19672,m,Germany,"system of a down,björk,portishead,the prodigy,..."
1720,19681,m,Netherlands,"the strokes,muse,led zeppelin,snow patrol,radi..."
1721,19685,m,Netherlands,"philip glass,rammstein,pink floyd,moby,blank &..."


In [18]:
artist_dummy_data = data_by_artists["artist"].str.get_dummies(',');

data_unp = pd.concat([data_by_artists, artist_dummy_data], axis = 1)
data_unp

,user,sex,country,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,1,f,Germany,"red hot chili peppers,the black dahlia murder,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31,f,Netherlands,"jamiroquai,tori amos,jazzanova,radiohead,st. g...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33,f,Germany,"death cab for cutie,tegan and sara,kimya dawso...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,f,Germany,"soundtrack,groove coverage,avril lavigne,the r...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51,f,Germany,"kate nash,arctic monkeys,lykke li,adele,tegan ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,19662,m,Germany,"blind guardian,in extremo,subway to sally,finn...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1719,19672,m,Germany,"system of a down,björk,portishead,the prodigy,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720,19681,m,Netherlands,"the strokes,muse,led zeppelin,snow patrol,radi...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1721,19685,m,Netherlands,"philip glass,rammstein,pink floyd,moby,blank &...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
frequent_itemsets = apriori(artist_dummy_data, min_support=0.02, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.057458,(3 doors down)
1,0.041207,(30 seconds to mars)
2,0.020313,(50 cent)
3,0.029019,(a perfect circle)
4,0.031921,(abba)
...,...,...
518,0.024376,"(system of a down, slipknot)"
519,0.028439,"(the offspring, system of a down)"
520,0.023796,"(the beatles, the doors)"
521,0.022055,"(the rolling stones, the beatles)"


## Правило 1

In [20]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(3 doors down),(linkin park),0.057458,0.142194,0.022635,0.393939,2.770439,0.014465,1.415380
1,(ac/dc),(die Ärzte),0.065003,0.143355,0.020894,0.321429,2.242192,0.011575,1.262425
2,(ac/dc),(metallica),0.065003,0.110273,0.020313,0.312500,2.833882,0.013145,1.294149
3,(ac/dc),(red hot chili peppers),0.065003,0.154962,0.023215,0.357143,2.304708,0.013142,1.314503
4,(air),(coldplay),0.087057,0.171793,0.023796,0.273333,1.591059,0.008840,1.139734
...,...,...,...,...,...,...,...,...,...
304,(system of a down),(the offspring),0.120720,0.068485,0.028439,0.235577,3.439822,0.020171,1.218585
305,(the doors),(the beatles),0.047591,0.123622,0.023796,0.500000,4.044601,0.017912,1.752757
306,(the rolling stones),(the beatles),0.048172,0.123622,0.022055,0.457831,3.703490,0.016099,1.616431
307,(the kooks),(the killers),0.094602,0.105049,0.034243,0.361963,3.445650,0.024305,1.402663


Просмотрим количество уникальных посылок и следствий

In [21]:
rules['antecedents'].value_counts()

(arctic monkeys)              17
(system of a down)            15
(muse)                        13
(foo fighters)                13
(the kooks)                   11
                              ..
(3 doors down)                 1
(marilyn manson)               1
(moby)                         1
(the prodigy)                  1
(rage against the machine)     1
Name: antecedents, Length: 68, dtype: int64

In [22]:
rules['consequents'].value_counts().head()

(die Ärzte)                27
(linkin park)              26
(red hot chili peppers)    24
(coldplay)                 24
(system of a down)         20
Name: consequents, dtype: int64

Сильных перекосов не обнаружено<br>

Найдем самую популярную группу и посмотрим какие еще группы можно рекомендовать ее фанатам


In [23]:
min = 0;
name = ""
arr = [[]]
for col in data_unp.columns[4:]:
    if(data_unp[col].sum() > min):
        min = data_unp[col].sum()
        name = col
print(name, " ", min)

coldplay   296


Составим правило с list > 1 и поддержкой > 0.02

In [24]:
rule1 = rules[((rules.lift>1) & (rules.support > 0.02))]
rule1 = rule1[(rule1['antecedents'] == {'coldplay'})]
rule1.sort_values(by = ['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
90,(coldplay),(jack johnson),0.171793,0.099826,0.037725,0.219595,2.199776,0.020575,1.153470
108,(coldplay),(the kooks),0.171793,0.094602,0.034823,0.202703,2.142679,0.018571,1.135583
15,(coldplay),(arctic monkeys),0.171793,0.099826,0.035403,0.206081,2.064405,0.018254,1.133836
98,(coldplay),(muse),0.171793,0.106790,0.037725,0.219595,2.056312,0.019379,1.144546
101,(coldplay),(radiohead),0.171793,0.117818,0.038305,0.222973,1.892524,0.018065,1.135330
103,(coldplay),(red hot chili peppers),0.171793,0.154962,0.047011,0.273649,1.765905,0.020390,1.163401


## Правило 2

Lift < 1 означает, что группы чаще встречаются по отдельности, чем вместе. Таких рекомендаций давать нельзя

In [25]:
rules_low_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.03)

In [26]:
rule2 = rules_low_conf.sort_values(by = 'lift').head(2);
rule2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
135,(rammstein),(coldplay),0.132327,0.171793,0.020313,0.153509,0.893566,-0.00242,0.978400
134,(coldplay),(rammstein),0.171793,0.132327,0.020313,0.118243,0.893566,-0.00242,0.984027


## Правило 3

In [27]:
frequent_itemsets_fpgrowth = fpgrowth(artist_dummy_data, min_support=0.01, use_colnames=True)
frequent_itemsets_fpgrowth

,support,itemsets
0,0.154962,(red hot chili peppers)
1,0.105049,(the killers)
2,0.099826,(jack johnson)
3,0.048172,(the rolling stones)
4,0.044689,(schandmaul)
...,...,...
2357,0.012188,"(godsmack, system of a down)"
2358,0.015670,"(a perfect circle, tool)"
2359,0.011027,"(a perfect circle, system of a down)"
2360,0.010447,"(incubus, a perfect circle)"


In [28]:
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.5)
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(schandmaul),(subway to sally),0.044689,0.055717,0.023796,0.532468,9.556683,0.021306,2.019717
1,"(subway to sally, schandmaul)",(in extremo),0.023796,0.052815,0.013929,0.585366,11.083356,0.012672,2.284388
2,"(subway to sally, in extremo)",(schandmaul),0.026698,0.044689,0.013929,0.521739,11.674760,0.012736,1.997467
3,"(schandmaul, in extremo)",(subway to sally),0.021474,0.055717,0.013929,0.648649,11.641892,0.012733,2.687575
4,"(subway to sally, system of a down)",(schandmaul),0.017992,0.044689,0.010447,0.580645,12.992878,0.009643,2.278048
...,...,...,...,...,...,...,...,...,...
242,(godsmack),(disturbed),0.020313,0.060360,0.012188,0.600000,9.940385,0.010962,2.349100
243,(godsmack),(koЯn),0.020313,0.069066,0.011027,0.542857,7.860024,0.009624,2.036419
244,(godsmack),(system of a down),0.020313,0.120720,0.012188,0.600000,4.970192,0.009736,2.198201
245,(a perfect circle),(tool),0.029019,0.048172,0.015670,0.540000,11.209880,0.014272,2.069192


Проверка на перекосы

In [29]:
rules['antecedents'].value_counts()

(arctic monkeys)              17
(system of a down)            15
(muse)                        13
(foo fighters)                13
(the kooks)                   11
                              ..
(3 doors down)                 1
(marilyn manson)               1
(moby)                         1
(the prodigy)                  1
(rage against the machine)     1
Name: antecedents, Length: 68, dtype: int64

In [30]:
rules['consequents'].value_counts().head()

(die Ärzte)                27
(linkin park)              26
(red hot chili peppers)    24
(coldplay)                 24
(system of a down)         20
Name: consequents, dtype: int64

In [31]:
rules_fpgrowth["antecedent_len"] = rules_fpgrowth["antecedents"].apply(lambda x: len(x))
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(schandmaul),(subway to sally),0.044689,0.055717,0.023796,0.532468,9.556683,0.021306,2.019717,1
1,"(subway to sally, schandmaul)",(in extremo),0.023796,0.052815,0.013929,0.585366,11.083356,0.012672,2.284388,2
2,"(subway to sally, in extremo)",(schandmaul),0.026698,0.044689,0.013929,0.521739,11.674760,0.012736,1.997467,2
3,"(schandmaul, in extremo)",(subway to sally),0.021474,0.055717,0.013929,0.648649,11.641892,0.012733,2.687575,2
4,"(subway to sally, system of a down)",(schandmaul),0.017992,0.044689,0.010447,0.580645,12.992878,0.009643,2.278048,2
...,...,...,...,...,...,...,...,...,...,...
242,(godsmack),(disturbed),0.020313,0.060360,0.012188,0.600000,9.940385,0.010962,2.349100,1
243,(godsmack),(koЯn),0.020313,0.069066,0.011027,0.542857,7.860024,0.009624,2.036419,1
244,(godsmack),(system of a down),0.020313,0.120720,0.012188,0.600000,4.970192,0.009736,2.198201,1
245,(a perfect circle),(tool),0.029019,0.048172,0.015670,0.540000,11.209880,0.014272,2.069192,1


Найдем правило с составной посылкой, наибольшей уверенностью и lift > 1 <br>
51% слушателей disturbed и koЯn также слушают менее популярную papa roach.

In [32]:
rules = rules_fpgrowth
rules3 = rules[((rules.lift > 10) & (rules.antecedent_len >= 2))]
rules3.sort_values(by = 'confidence', ascending = True).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
202,"(koЯn, disturbed)",(papa roach),0.026117,0.048172,0.013349,0.511111,10.610174,0.012091,1.946921,2


In [33]:
data_unp['koЯn'].value_counts()

0    1604
1     119
Name: koЯn, dtype: int64

In [34]:
data_unp['disturbed'].value_counts()

0    1619
1     104
Name: disturbed, dtype: int64

In [35]:
data_unp['papa roach'].value_counts()

0    1640
1      83
Name: papa roach, dtype: int64

## Правило 4

In [36]:
data_by_artists_sex_dummies = pd.concat([data_by_artists['sex'].str.get_dummies(','),
                             data_by_artists['artist'].str.get_dummies(',')], axis = 1)
data_by_artists_sex_dummies

,f,m,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1719,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1721,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
frequent_itemsets_fpgrowth2 = fpgrowth(data_by_artists_sex_dummies, min_support=0.01, use_colnames=True)

In [38]:
rules_fpgrowth2 = association_rules(frequent_itemsets_fpgrowth2, metric="lift", min_threshold=0.3)
rules_fpgrowth2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(f),(red hot chili peppers),0.233894,0.154962,0.043529,0.186104,1.200965,0.007284,1.038263
1,(red hot chili peppers),(f),0.154962,0.233894,0.043529,0.280899,1.200965,0.007284,1.065366
2,(m),(red hot chili peppers),0.766106,0.154962,0.111434,0.145455,0.938645,-0.007284,0.988874
3,(red hot chili peppers),(m),0.154962,0.766106,0.111434,0.719101,0.938645,-0.007284,0.832664
4,(coldplay),(red hot chili peppers),0.171793,0.154962,0.047011,0.273649,1.765905,0.020390,1.163401
...,...,...,...,...,...,...,...,...,...
10773,(m),(taking back sunday),0.766106,0.016251,0.010447,0.013636,0.839123,-0.002003,0.997349
10774,(m),(ladytron),0.766106,0.017411,0.015670,0.020455,1.174773,0.002331,1.003107
10775,(ladytron),(m),0.017411,0.766106,0.015670,0.900000,1.174773,0.002331,2.338944
10776,(m),(burial),0.766106,0.011608,0.011027,0.014394,1.240038,0.002135,1.002827


Список групп, которые не стоит рекомендовать мужчинам (lift < 1)

In [39]:
rules_fpgrowth2["consequents_len"] = rules_fpgrowth2["consequents"].apply(lambda x: len(x))
rule4 = rules_fpgrowth2[(rules_fpgrowth2['antecedents'] == {'m'})
    & (rules_fpgrowth2['consequents'].map(lambda x: (('f' not in x) & ('m' not in x))))
                        & (rules_fpgrowth2['consequents_len'] == 1)
                       ].sort_values(by='lift', ascending=True)
rule4[(rule4.lift < 0.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
6274,(m),(regina spektor),0.766106,0.027858,0.011608,0.015152,0.543876,-0.009735,0.987098,1
2124,(m),(paramore),0.766106,0.037145,0.015670,0.020455,0.550675,-0.012786,0.982962,1
5689,(m),(pink),0.766106,0.037725,0.016251,0.021212,0.562284,-0.012651,0.983129,1
10130,(m),(britney spears),0.766106,0.034823,0.015090,0.019697,0.565631,-0.011588,0.984570,1
3241,(m),(lily allen),0.766106,0.027858,0.012188,0.015909,0.571070,-0.009154,0.987858,1
3184,(m),(kate nash),0.766106,0.040627,0.018572,0.024242,0.596710,-0.012552,0.983209,1
6268,(m),(panic at the disco),0.766106,0.027858,0.012768,0.016667,0.598264,-0.008574,0.988619,1


Список групп, которые можно рекомендовать женщинам (lift > 1)

In [40]:
rule5 = rules_fpgrowth2[(rules_fpgrowth2['antecedents'] == {'f'})
    & (rules_fpgrowth2['consequents'].map(lambda x: (('f' not in x) & ('m' not in x))))
                        & (rules_fpgrowth2['consequents_len'] == 1)
                       ].sort_values(by='lift', ascending=False)
rule5[(rule5.lift > 1)]
rule5.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
2184,(f),(the fray),0.233894,0.024376,0.017411,0.074442,3.053882,0.011710,1.054092,1
2197,(f),(tegan and sara),0.233894,0.022635,0.014510,0.062035,2.740663,0.009215,1.042006,1
2198,(f),(paolo nutini),0.233894,0.020894,0.013349,0.057072,2.731527,0.008462,1.038368,1
6272,(f),(regina spektor),0.233894,0.027858,0.016251,0.069479,2.494003,0.009735,1.044728,1
2120,(f),(paramore),0.233894,0.037145,0.021474,0.091811,2.471735,0.012786,1.060193,1


## Правило 5
Следующие группы слушают мужчины, маловероятно, что их можно рекомендовать женщинам

In [41]:
rule6 = rules_fpgrowth2[(rules_fpgrowth2['antecedents'].map(lambda x: (('f' not in x) & ('m' not in x))))
                        &(rules_fpgrowth2['consequents'] == {'m'})
                        & (rules_fpgrowth2['consequents_len'] == 1)
                       ].sort_values(by='lift', ascending=False)
rule6[(rule6.lift > 1)]
rule6.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
6537,"(the prodigy, limp bizkit)",(m),0.010447,0.766106,0.010447,1.0,1.305303,0.002443,inf,1
10495,(underworld),(m),0.016831,0.766106,0.016831,1.0,1.305303,0.003937,inf,1
10442,(sepultura),(m),0.015670,0.766106,0.015670,1.0,1.305303,0.003665,inf,1
10425,"(arch enemy, in flames)",(m),0.014510,0.766106,0.014510,1.0,1.305303,0.003394,inf,1
2096,"(the offspring, seeed)",(m),0.010447,0.766106,0.010447,1.0,1.305303,0.002443,inf,1
